In [1]:
#wigets and display widgets
import ipywidgets as wg
from ipywidgets import interact, interactive, fixed, interact_manual
from IPython.display import display

#NLP and clustering
import spacy
from clustering import Clustering
from topic_determination import Topic_determination

In [2]:
#!pip install azure-ai-textanalytics

In [3]:
#!python -m spacy download de_core_news_lg

In [4]:
nlp = spacy.load('de_core_news_lg')
clustering = Clustering(nlp)
topic_determination = Topic_determination()
metadata = []
topics = []
blacklist = []
df = None

In [22]:
def best_topic_to_question(x):
    global topics
    global blacklist
    global df
    if x == "ja":
        print(topics[0] in df.loc[df['Topics']])
        return "\n".join(df.loc[df['Topics'] == topics[0]]["d115Name"].tolist())
    elif x == "nein":
        blacklist.append(topics[0])
        topics.pop(0)
        if topics:
            return "Geht es bei ihrem Anliegen um " + topics[0] + "?"
        return "Sorry nothing there."
    else:
        global metadata
        metadata.append(x)
        df = clustering.run("antrag", cluster_by="ssdsLemma", do_preprocessing=True, query_max_elems=2000, cluster_eps=1.3, cluster_min_samples=1)
        #df = clustering.run(" ".join(metadata), cluster_by="ssdsLemma", do_preprocessing=True, query_max_elems=2000, cluster_eps=1.3, cluster_min_samples=1)
    
        df_res=topic_determination.run(df,1,"ssdsLemma","c09dde90c9f34da3b69065aabdb735c8")
        for topic in df['Topics']:
            topics.append(topic[0])
        return "Geht es bei ihrem Anliegen um " + topics[0] + "?"

In [23]:
interact(lambda x: best_topic_to_question(x), x="Antrag");

interactive(children=(Text(value='Antrag', description='x'), Output()), _dom_classes=('widget-interact',))

In [87]:
#name = wg.Text("Hello")
#question = wg.Text("antrag")
# widgets.HTML(
#     value="Hello <b>User</b> please enter your questions here!",
#     placeholder='',
#     description='',
# )
#display(question)

In [88]:
#widgetLink = widgets.jslink((floatTextBox, 'value'), (floatSlider, 'value'))

In [89]:
# widgets.Textarea(
#     value='Hello World',
#     placeholder='Type something',
#     description='String:',
#     disabled=False
# )